In [4]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.python.client import device_lib
#check if gpu available
def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]
print(get_available_devices()) 
# gpu memory
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

['/device:CPU:0', '/device:GPU:0']


# Load dataset

In [5]:
from tensorflow.keras.utils import to_categorical  

#load cifar-10 dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
#one hot encoding
y_test=to_categorical(y_test, num_classes=10)
y_train=to_categorical(y_train, num_classes=10)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(50000, 32, 32, 3)
(50000, 10)
(10000, 32, 32, 3)
(10000, 10)


### Validation data

In [6]:
#shuffle data and split data to generate validation data
shuffler = np.random.permutation(len(x_train))
x_train = x_train[shuffler]
y_train = y_train[shuffler]
 

#validation data
x_val=x_train[0:2000]
y_val=y_train[0:2000]

x_train=x_train[2000:]
y_train=y_train[2000:]


In [7]:
batch_size=10
epochs=3
base_learning_rate=0.0001

# InceptionV3

![image](./figures/inceptionv3.png)

In [ ]:
inception_v3_model = tf.keras.applications.inception_v3.InceptionV3(
    include_top=True, weights=None, classes=10, input_shape=(150,150,3)
)

In [ ]:
help(tf.keras.applications.inception_v3.InceptionV3)

In [ ]:
inception_v3_model.summary()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import scipy

inception_v3_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),loss='categorical_crossentropy', metrics = ['acc'])

train_datagen = ImageDataGenerator(
        horizontal_flip=True,
        width_shift_range=4, #+-4 pixel shift max
        height_shift_range=4,#+-4 pixel shift max
        cval=0, #zero padding
        fill_mode='constant' #zero padding
        )

# train_generator = train_datagen.flow(x_train, y_train, batch_size=batch_size)
# step_size_train=train_generator.n//train_generator.batch_size

inputs = tf.keras.Input(shape=(32, 32, 3))
x = tf.keras.applications.InceptionV3.re
x = inception_v3_model(x)
x=GlobalAveragePooling2D()(x)
preds=Dense(10,activation='softmax')(x)
inception_v3_model_cifar10=Model(inputs=inputs,outputs=preds)
inception_v3_model_cifar10.summary()

Size of CIFAR (32x32) is incompatible with InceptionV3, if we fed this input to the first convolutional layers in InceptionV3, we'd end up with no pixels at the end (because of the max pooling operations). We resize the input to 299x299.

In [ ]:
x_train_reshaped = tf.image.resize(x_train, (150,150))

x_val_reshaped =  tf.image.resize(x_val, (150,150))

In [ ]:
plt.imshow(x_train_reshaped[0]/255)
plt.show()
plt.imshow(x_train_reshaped[1]/255)
plt.show()
plt.imshow(x_train_reshaped[2]/255)
plt.show()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import scipy

inception_v3_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),loss='categorical_crossentropy', metrics = ['acc'])

train_datagen = ImageDataGenerator(
        horizontal_flip=True,
        width_shift_range=4, #+-4 pixel shift max
        height_shift_range=4,#+-4 pixel shift max
        cval=0, #zero padding
        fill_mode='constant' #zero padding
        )

# train_generator = train_datagen.flow(x_train, y_train, batch_size=batch_size)
# step_size_train=train_generator.n//train_generator.batch_size



# x_train= x_train[:5000]
# y_train= y_train[:5000]


history=inception_v3_model.fit(x=x_train_reshaped, y=y_train,
                   epochs = 30,
                   validation_data=(x_val_reshaped, y_val))

# Adding SE layers

### This is what we had before: normal resnet50

In [ ]:
keras.utils.plot_model(inception_v3_model, to_file='inception_v3_model.png',show_shapes=True)


In [ ]:
import pydot

### List of layers where SE is going to be added after the output

(Check resnet50.png for more information)

In [ ]:
l=[l.name for l in resnet50.layers]
list_se=[]
for i,name in enumerate(l):
    if(i+1<len(l) and 'add' in l[i+1]):
        print(name)
        list_se.append(name)


In [ ]:
###### Not used #######
#custom SE Layer (not working)
from tensorflow.python.keras.layers import GlobalAveragePooling2D
class SE_layer(tf.keras.layers.Layer):
    def __init__(self, ratio=16,initializer="he_normal", **kwargs):
        super(SE_layer, self).__init__(**kwargs)
        self.initializer = keras.initializers.get(initializer)
        self.ratio=ratio

    def build(self, input_shape):
        out_dim = input_shape[-1]
    def call(self, inputs): 
        print(inputs.shape[-1])
        out_dim=inputs.shape[-1]
        F_sq = GlobalAveragePooling2D()(inputs) #squeeze
        F_ex = Dense(out_dim / self.ratio,activation='relu')(F_sq)
        F_ex = Dense(out_dim,activation='sigmoid')(F_ex)
        F_ex = tf.reshape(F_ex, [-1,1,1,out_dim])
        F_scale = inputs * F_ex
        return F_scale

    def get_config(self):
        # Implement get_config to enable serialization.
        base_config = super(SE_layer, self).get_config()
        config = {"initializer": keras.initializers.serialize(self.initializer)}
        return dict(list(base_config.items()) + list(config.items()))

### Adding SE layers before 'add' layers

In [ ]:
####### add SE layers to model ##########
# input: model
#        l: list l of layer names where SE layers are going to go after
#        ratio : SE ratio
#        modelname: name of new model
#
# Output: modified model with SE layers at specified locations
# See https://stackoverflow.com/questions/49492255/how-to-replace-or-insert-intermediate-layer-in-keras-model to understand the function
##########################################

def add_SE(model, l,ratio,modelname=None):
    se_num =1
    # Auxiliary dictionary to describe the network graph
    network_dict = {'input_layers_of': {}, 'new_output_tensor_of': {}}

    # Set the input layers of each layer
    for layer in model.layers:
        for node in layer._outbound_nodes:
            layer_name = node.outbound_layer.name
            if layer_name not in network_dict['input_layers_of']:
                network_dict['input_layers_of'].update(
                        {layer_name: [layer.name]})
            else:
                network_dict['input_layers_of'][layer_name].append(layer.name)

    # Set the output tensor of the input layer
    network_dict['new_output_tensor_of'].update(
            {model.layers[0].name: model.input})

    # Iterate over all layers after the input
    model_outputs = []
    for layer in model.layers[1:]:

        # Determine input tensors
        layer_input = [network_dict['new_output_tensor_of'][layer_aux] 
                for layer_aux in network_dict['input_layers_of'][layer.name]]
        if len(layer_input) == 1:
            layer_input = layer_input[0]

        # Insert SE layers if name matches the argument list
        if layer.name in l:
            #input of SE
            x = layer(layer_input)
            
            ### adding SE layers
            out_dim=layer.output_shape[-1]
            #squeeze layer
            F_sq = GlobalAveragePooling2D(name='SE'+str(se_num)+'_global_avg')(x) #squeeze
            #Excitation with 2 fully connected layers
            F_ex = Dense(out_dim / ratio,activation='relu', name='SE'+str(se_num)+'_dense_relu')(F_sq)
            F_ex = Dense(out_dim,activation='sigmoid', name='SE'+str(se_num)+'_dense_sig')(F_ex)
            #Output : rescaling
            F_ex = tf.reshape(F_ex, [-1,1,1,out_dim])
            x=keras.layers.multiply([x,F_ex], name='SE'+str(se_num)+'_scaling')
            
            se_num+=1 # for naming
            print('added SE after ',layer.name)
        else:
            x = layer(layer_input)

        # Set new output tensor (the original one, or the one of the inserted
        # layer)
        network_dict['new_output_tensor_of'].update({layer.name: x})

        # Renaming model
        if layer.name in model.output_names:
            model_outputs.append(x)
        my_final_model= Model(inputs=model.inputs, outputs=model_outputs)
        if (modelname!=None):
            my_final_model._name = modelname
        else:
            my_final_model._name='SE_'+model._name
    return my_final_model

SE_ResNet50 = ResNet50(include_top=False, weights=None)
def SE_factory(ratio):
    return SE_layer(ratio=ratio)

SE_ResNet50 = add_SE(SE_ResNet50, list_se,ratio=16)
#SE_ResNet50.save('temp.h5')
#SE_ResNet50 = keras.models.load_model('temp.h5',custom_objects={'SE_layer': SE_layer})
SE_ResNet50.summary()
keras.utils.plot_model(SE_ResNet50, to_file='test.png',show_shapes=True)

### Adding topo layer, and input preprocessing (different resolution)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.python.keras.layers import Dense, GlobalAveragePooling2D

inputs = tf.keras.Input(shape=(32, 32, 3))
# Preprocess for ResNet50 input size
x = preprocess_input(inputs)
# ResNet50
x = SE_ResNet50(x)
# Add top layer again
x=GlobalAveragePooling2D()(x)
#10 classes for cifar 10
preds=Dense(10,activation='softmax')(x)
SE_resnet50_cifar10=Model(inputs=inputs,outputs=preds)
SE_resnet50_cifar10.summary()


# Todo : train model with custom loop, adaptive learning rate, saving model every k epochs etc

# Top n accuracy

In [ ]:
def top_n_acc(pred, targets,n=1):
    acc=np.mean(tf.keras.metrics.top_k_categorical_accuracy(targets, pred, k=n).numpy())
    print("top",n,'accuracy:',acc,sep=' ')
    return acc

In [ ]:
preds=resnet50_cifar10.predict(x_test)
top_n_acc(preds,y_test,1)

# Recreating Inception model

**Source:**  
• https://github.com/pytorch/vision/blob/6db1569c89094cf23f3bc41f79275c45e9fcb3f3/torchvision/models/inception.py  
• https://github.com/tensorflow/models/blob/master/research/slim/nets/inception_v3.py  
• https://www.youtube.com/watch?v=ekixid21Q5U  

All code is our own. We did write all of our code, and only used the above links as references on how the InceptionV3 model works and how to parametrize it.


In [41]:
from utils.SE_inception_v3 import inception_v3_custom

se_network=True # use or not an SE network
aux_classifier=True # use or not an auxiliary classifier; it is an intermediatary classifier in the model that is supposed to help reduce overfitting
data_base='cifar-10' # the database that we'll use
ratio = 16 # the r ratio of our SE network; authors recommand 16

inception_v3 = inception_v3_custom(aux_classifier=aux_classifier, se_network=se_network, data_base= data_base, ratio = ratio)
inception_v3_custom_model = inception_v3.Model(name = 'inception_v3_custom_model')

inception_v3_custom_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.006), loss='categorical_crossentropy', metrics = ['acc'])
history=inception_v3_custom_model.fit(x=x_train, y=y_train,
                   epochs = 15,
                   validation_data=(x_val, y_val))


Epoch 1/15
1500/1500 [==============================] - 129s 79ms/step - loss: 6.1416 - auxiliary_classifier_loss: 3.9121 - prediction_layer_loss: 2.2295 - auxiliary_classifier_acc: 0.1857 - prediction_layer_acc: 0.1786 - val_loss: 4.2622 - val_auxiliary_classifier_loss: 2.0917 - val_prediction_layer_loss: 2.1705 - val_auxiliary_classifier_acc: 0.2970 - val_prediction_layer_acc: 0.3060
Epoch 2/15
 101/1500 [=>............................] - ETA: 1:46 - loss: 3.6020 - auxiliary_classifier_loss: 1.8003 - prediction_layer_loss: 1.8017 - auxiliary_classifier_acc: 0.3543 - prediction_layer_acc: 0.3430

KeyboardInterrupt: 

In [5]:
def Conv2D_Stack(input, filters, kernel_size = 1, strides = 1, padding = 'same', activation = 'relu') :
    #A Conv2D layer followed by a Batch normalization and an activation
    input_shape = tf.shape(input)
    output = Conv2D(filters=filters, kernel_size= kernel_size, strides = strides, padding = padding, input_shape=input_shape[1:])(input)
    output = BatchNormalization()(output)
    output = Activation(activation)(output)
    return output

In [6]:
#TODO: Inception Block A
def block_A(input, number = 1):
    #number = number of the block A in inception (in total: 3 blocks)

    filters_branch_4 = 64 - (number == 1) * 32 #32 filters for first block A, 64 for the others

    branch_1 = Conv2D_Stack(input, filters = 64, kernel_size = (1,1), activation = 'relu')

    branch_2 = Conv2D_Stack(input, filters = 48, kernel_size = (1,1), activation = 'relu')
    branch_2 = Conv2D_Stack(branch_2, filters = 64, kernel_size = (5,5), activation = 'relu')

    branch_3 = Conv2D_Stack(input, filters = 64 , kernel_size = (1,1), activation = 'relu')
    branch_3 = Conv2D_Stack(branch_3, filters = 96, kernel_size = (3,3), activation = 'relu')
    branch_3 = Conv2D_Stack(branch_3, filters = 96, kernel_size = (3,3), activation = 'relu')

    branch_4 = AvgPool2D(pool_size = (3,3), strides = (1,1), padding = 'same')(input)
    branch_4 = Conv2D_Stack(branch_4, filters = filters_branch_4, kernel_size = (1,1), activation = 'relu')

    output = tf.concat(axis=3, values=[branch_1, branch_2, branch_3, branch_4])
    
    return output

In [7]:
z#TODO: Inception Block B
#Note: At each use of Block B, Tensorflow implementaton might take values slightly different for the kernel
#size, I might take that into consideration later on

def block_B(input, number = 1):

    filters_inter = (number == 1) * 128 + (number in (2,3) ) * 160 + (number == 4) * 192 #Intermediate Conv2D layers of first block B have 128 filters, then 160, 160, 192 for 2nd, 3rd and 4th instance of block B

    branch_1 = Conv2D_Stack(input, filters = 192, kernel_size = (1,1), activation = 'relu')

    branch_2 = AvgPool2D(pool_size = (3,3), strides = (1,1), padding = 'same')(input)
    branch_2 = Conv2D_Stack(branch_2, filters = 192, kernel_size = (1,1), activation = 'relu')

    branch_3 = Conv2D_Stack(input, filters = filters_inter, kernel_size = (1,1), activation = 'relu')
    branch_3 = Conv2D_Stack(branch_3, filters = filters_inter, kernel_size = (1,7), activation = 'relu')
    branch_3 = Conv2D_Stack(branch_3, filters = 192, kernel_size = (7,1), activation = 'relu')

    branch_4 = Conv2D_Stack(input, filters = filters_inter , kernel_size = (1,1), activation = 'relu')
    branch_4 = Conv2D_Stack(branch_4, filters = filters_inter, kernel_size = (7,1), activation = 'relu')
    branch_4 = Conv2D_Stack(branch_4, filters = filters_inter, kernel_size = (1,7), activation = 'relu')
    branch_4 = Conv2D_Stack(branch_4, filters = filters_inter, kernel_size = (7,1), activation = 'relu')
    branch_4 = Conv2D_Stack(branch_4, filters = 192, kernel_size = (1,7), activation = 'relu')

    output = tf.concat(axis=3, values=[branch_1, branch_2, branch_3, branch_4])
    
    return output

In [8]:
#TODO: Inception Block C
def block_C(input):
    #block_C is called 2 times in incetpion, with the exact same parameters

    branch_1 = Conv2D_Stack(input, filters = 320, kernel_size = (1,1), activation = 'relu')

    branch_2 = Conv2D_Stack(input, filters = 384, kernel_size = (1,1), activation = 'relu')
    branch_2_a = Conv2D_Stack(branch_2, filters = 384, kernel_size = (1,3), activation = 'relu')
    branch_2_b = Conv2D_Stack(branch_2, filters = 384, kernel_size = (3,1), activation = 'relu')

    branch_3 = Conv2D_Stack(input, filters = 448 , kernel_size = (1,1), activation = 'relu')
    branch_3 = Conv2D_Stack(branch_3, filters = 384, kernel_size = (3,3), activation = 'relu')
    branch_3_a = Conv2D_Stack(branch_3, filters = 384, kernel_size = (3,1), activation = 'relu')
    branch_3_b = Conv2D_Stack(branch_3, filters = 384, kernel_size = (1,3), activation = 'relu')

    branch_4 = AvgPool2D(pool_size = (3,3), strides = (1,1), padding = 'same')(input)
    branch_4 = Conv2D_Stack(branch_4, filters = 192, kernel_size = (1,1), activation = 'relu')

    output = tf.concat(axis=3, values=[branch_1, branch_2_a, branch_2_b, branch_3_a, branch_3_b, branch_4])

    return output

In [9]:
#TODO: Inception Reduction A
def reduction_block_A(input):
    branch_1 = MaxPool2D(pool_size = (3,3), strides = (2,2), padding = 'valid')(input)

    branch_2 = Conv2D_Stack(input, kernel_size = (3,3), filters = 384, strides = 2, padding = 'valid', activation = 'relu')

    branch_3 = Conv2D_Stack(input, filters = 64, kernel_size = (1,1), activation = 'relu')
    branch_3 = Conv2D_Stack(branch_3, filters = 96, kernel_size = (3,3), activation = 'relu')
    branch_3 = Conv2D_Stack(branch_3, filters = 96, kernel_size = (3,3), strides = 2, padding = 'valid', activation = 'relu')

    output = tf.concat(axis=3, values=[branch_1, branch_2, branch_3])

    return output

In [10]:
#TODO: Inception Reduction B
def reduction_block_B(input):
    branch_1 = MaxPool2D(pool_size = (3,3), strides = (2,2), padding = 'valid')(input)

    branch_2 = Conv2D_Stack(input, kernel_size = (1,1), filters = 192, activation = 'relu')
    branch_2 = Conv2D_Stack(branch_2, kernel_size = (3,3), filters = 320, strides = 2, padding = 'valid', activation = 'relu')

    branch_3 = Conv2D_Stack(input, filters = 192, kernel_size = (1,1), activation = 'relu')
    branch_3 = Conv2D_Stack(branch_3, filters = 192, kernel_size = (1,7), activation = 'relu')
    branch_3 = Conv2D_Stack(branch_3, filters = 192, kernel_size = (7,1), activation = 'relu')
    branch_3 = Conv2D_Stack(branch_3, filters = 192, kernel_size = (3,3), strides = 2, padding = 'valid', activation = 'relu')

    output = tf.concat(axis=3, values=[branch_1, branch_2, branch_3])

    return output

In [11]:
#TODO: Inception forward
input = tf.keras.layers.Input(shape = (32, 32, 3))  ####### normally  would be input = tf.keras.layers.Input(shape = (299, 299, 3))
n_classes = 10

######### CUSTOM 

x = tf.image.resize(input, size = (200, 200))
x = Conv2D_Stack(x, kernel_size = (3,3), filters = 32, strides = 2, padding = 'valid', activation = 'relu')

# x = tf.keras.layers.Flatten()(input)
# x = tf.keras.layers.Dense(299 * 299 * 3, activation = 'relu')(x)
# x = tf.reshape(x, (-1, 299, 299, 3))
# print(tf.shape(x))
# x = Conv2D_Stack(x, kernel_size = (3,3), filters = 32, strides = 2, padding = 'valid', activation = 'relu')

#####END CUSTOM

#x = Conv2D_Stack(input, kernel_size = (3,3), filters = 32, strides = 2, padding = 'valid', activation = 'relu')
x = Conv2D_Stack(x, kernel_size = (3,3), filters = 32, strides = 1, padding = 'valid', activation = 'relu')
x = Conv2D_Stack(x, kernel_size = (3,3), filters = 64, strides = 1, padding = 'valid', activation = 'relu')

x = MaxPool2D(pool_size = (3,3), strides = (2,2), padding = 'valid')(x)

x = Conv2D_Stack(x, kernel_size = (1,1), filters = 80, strides = 1, padding = 'valid', activation = 'relu')

x = Conv2D_Stack(x, kernel_size = (3,3), filters = 192, strides = 1, padding = 'valid', activation = 'relu')

x = MaxPool2D(pool_size = (3,3), strides = (2,2), padding = 'valid')(x)

x = block_A(x, number = 1)
x = block_A(x, number = 2)
x = block_A(x, number = 3)

x = reduction_block_A(x)

x = block_B(x, number = 1)
x = block_B(x, number = 2)
x = block_B(x, number = 3)
x = block_B(x, number = 4)

###auxiliary classifier for better performance (only useful for backward propagation)

out1 = AvgPool2D(pool_size = (5,5), strides = 3, padding = 'same')(x)
out1 = Conv2D_Stack(out1, filters = 128, kernel_size = 1, strides = 1, padding = 'same')
out1 = Conv2D_Stack(out1, filters = 768, kernel_size = (5,5), strides = 1, padding = 'same')
out1 = tf.keras.layers.Flatten()(out1)
initializer = tf.keras.initializers.TruncatedNormal(0.001)
out1 = tf.keras.layers.Dense(n_classes, activation = 'softmax', name = 'auxiliary_classifier', kernel_initializer = initializer)(out1)

###


x = reduction_block_B(x)
x = block_C(x)
x = block_C(x)

x = tf.keras.layers.GlobalAveragePooling2D(name = 'average_pooling')(x)
x = tf.keras.layers.Dropout(rate = 0.2)(x)
out2 = tf.keras.layers.Dense(n_classes, activation = 'softmax', name = 'prediction_layer')(x)
tf.shape(out2)

<KerasTensor: shape=(2,) dtype=int32 inferred_value=[None, 10] (created by layer 'tf.compat.v1.shape_96')>

In [15]:
# inception_v3_custom_model = tf.keras.models.Model(input, out2, name = 'inception_v3_custom')
inception_v3_custom_model = tf.keras.models.Model(input, [out1, out2], name = 'inception_v3_custom')
inception_v3_custom_model.summary()


Model: "inception_v3_custom"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
tf.image.resize (TFOpLambda)    (None, 200, 200, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 99, 99, 32)   896         tf.image.resize[0][0]            
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 99, 99, 32)   128         conv2d[0][0]                     
________________________________________________________________________________

In [13]:
tf.keras.utils.plot_model(inception_v3_custom_model, to_file='inception_v3_custom_model2.png',show_shapes=True)

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [16]:
inception_v3_custom_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.003),loss='categorical_crossentropy', metrics = ['acc'])
history=inception_v3_custom_model.fit(x=x_train, y=y_train,
                   epochs = 15,
                   validation_data=(x_val, y_val))

Epoch 1/15
1500/1500 [==============================] - 130s 79ms/step - loss: 0.1977 - auxiliary_classifier_loss: 0.0928 - prediction_layer_loss: 0.1049 - auxiliary_classifier_acc: 0.9677 - prediction_layer_acc: 0.9654 - val_loss: 1.2887 - val_auxiliary_classifier_loss: 0.6597 - val_prediction_layer_loss: 0.6290 - val_auxiliary_classifier_acc: 0.8435 - val_prediction_layer_acc: 0.8505
Epoch 2/15
1500/1500 [==============================] - 116s 77ms/step - loss: 0.1288 - auxiliary_classifier_loss: 0.0578 - prediction_layer_loss: 0.0710 - auxiliary_classifier_acc: 0.9799 - prediction_layer_acc: 0.9766 - val_loss: 1.5824 - val_auxiliary_classifier_loss: 0.8210 - val_prediction_layer_loss: 0.7614 - val_auxiliary_classifier_acc: 0.8380 - val_prediction_layer_acc: 0.8440
Epoch 3/15
1500/1500 [==============================] - 116s 77ms/step - loss: 0.1050 - auxiliary_classifier_loss: 0.0469 - prediction_layer_loss: 0.0581 - auxiliary_classifier_acc: 0.9834 - prediction_layer_acc: 0.9803 - 

In [ ]:
inception_v3_custom_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),loss='categorical_crossentropy', metrics = ['acc'])
history=inception_v3_custom_model.fit(x=x_train, y=y_train,
                   epochs = 30,
                   validation_data=(x_val, y_val))

Epoch 1/30
1500/1500 [==============================] - 204s 114ms/step - loss: 4.6286 - auxiliary_classifier_loss: 2.8303 - prediction_layer_loss: 1.7983 - auxiliary_classifier_acc: 0.3750 - prediction_layer_acc: 0.3555 - val_loss: 2.8479 - val_auxiliary_classifier_loss: 1.3312 - val_prediction_layer_loss: 1.5167 - val_auxiliary_classifier_acc: 0.5425 - val_prediction_layer_acc: 0.4645
Epoch 2/30
1500/1500 [==============================] - 191s 128ms/step - loss: 2.6847 - auxiliary_classifier_loss: 1.4450 - prediction_layer_loss: 1.2397 - auxiliary_classifier_acc: 0.5597 - prediction_layer_acc: 0.5550 - val_loss: 2.6191 - val_auxiliary_classifier_loss: 1.4456 - val_prediction_layer_loss: 1.1735 - val_auxiliary_classifier_acc: 0.6170 - val_prediction_layer_acc: 0.5865
Epoch 3/30
1500/1500 [==============================] - 163s 108ms/step - loss: 2.1197 - auxiliary_classifier_loss: 1.1509 - prediction_layer_loss: 0.9688 - auxiliary_classifier_acc: 0.6568 - prediction_layer_acc: 0.6575